In [ ]:
from IPython.display import Markdown, display

In [ ]:
def printmd(string):
    display(Markdown(string))

### AreTomo Alignment

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import yaml

In [ ]:
aretomo_md_file = f"{proj_name}_aretomo_mdout.yaml"

with open(aretomo_md_file, "r") as f:
    md = yaml.load(f.read(), Loader=yaml.FullLoader)

ts_list = md["process_list"]
stat_files = md["aretomo_align_stats"]

In [ ]:
def get_stats(file_in):
    with open(file_in, "r") as f:
        lines = f.readlines()
        
    header = lines[2].strip().split()[1:]
    df = pd.read_csv(file_in, sep="\s+", comment="#", names=header)
    df["Abs_shift"] = np.sqrt(df.TX**2 + df.TY**2)
    
    return np.mean(df.Abs_shift), np.std(df.Abs_shift)

In [ ]:
mean_list = []
std_list = []

for idx, curr_file in enumerate(stat_files.values()):
    mean, std = get_stats(curr_file)
    mean_list.append(mean)
    std_list.append(std)
    
at_df = pd.DataFrame(columns=["Tilt series", "Mean shift (px)", "Shift s.d. (px)"])
at_df["Tilt series"] = ts_list
at_df["Mean shift (px)"] = mean_list
at_df["Shift s.d. (px)"] = std_list

In [ ]:
at_df

In [ ]:
plt.figure(figsize=(15,5))
ax = sns.barplot(data=at_df,
                 x="Tilt series",
                 y="Mean shift (px)",
                 color="cornflowerblue"
                )

ax.errorbar(x=at_df.index-1,
            y=at_df["Mean shift (px)"],
            yerr=at_df["Shift s.d. (px)"], fmt="none", c="k")

plt.gca().set_ylim(bottom=0)